In [2]:
import pandas as pd

In [3]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [4]:
PATH = "https://storage.yandexcloud.net/data-ice-lake-04/messager-data/analytics/cleaned-events/event_type%3Dreaction/date%3D2022-01-16/part-00066-c8b1c6b7-6df1-40a9-baa2-326562d1fc66.c000.gz.parquet"

In [5]:
df = pd.read_parquet(PATH)
df.head()

,admins,channel_id,datetime,media_type,media_src,message,message_channel_to,message_from,message_group,message_id,message_to,message_ts,reaction_from,reaction_type,subscription_channel,subscription_user,tags,user
0,None,NaN,2022-01-16 15:04:52,None,None,None,NaN,NaN,NaN,1115511,NaN,NaT,154022,dislike,NaN,None,None,None
1,None,NaN,2022-01-16 10:28:53,None,None,None,NaN,NaN,NaN,1101399,NaN,NaT,82319,like,NaN,None,None,None
2,None,NaN,2022-01-16 14:56:16,None,None,None,NaN,NaN,NaN,1101399,NaN,NaT,133394,like,NaN,None,None,None
3,None,NaN,2022-01-16 17:25:22,None,None,None,NaN,NaN,NaN,1101399,NaN,NaT,81644,dislike,NaN,None,None,None
4,None,NaN,2022-01-16 19:36:32,None,None,None,NaN,NaN,NaN,1093426,NaN,NaT,42021,like,NaN,None,None,None


In [6]:
df.reaction_type.unique()

array(['dislike', 'like'], dtype=object)

In [10]:
df.notna().sum()

admins                     0
channel_id                 0
datetime                1615
media_type                 0
media_src                  0
message                    0
message_channel_to         0
message_from               0
message_group              0
message_id              1615
message_to                 0
message_ts                 0
reaction_from           1615
reaction_type           1615
subscription_channel       0
subscription_user          0
tags                       0
user                       0
dtype: int64

In [9]:
df[df.tags.notna()].shape

(0, 18)